In [1]:
import numpy as np

def pieceToVector(piece: str, isBlackTurn: bool = False):
    pieceMap = {
        "r": 0,
        "n": 1,
        "b": 2,
        "q": 3,
        "k": 4,
        "p": 5
    }

    vector = np.zeros(7)
    vector[pieceMap[piece.lower()]] = 1

    if ((not isBlackTurn) and piece.isupper()) or (isBlackTurn and piece.islower()):
        vector[6] = 1

    return vector


# if the board is black need to flip values as if it is in the perspective of white
# for move generation 
# for evaluation this does not matter
def fenToMatrix(fen: str, isBlackTurn: bool = False):
    matrix = np.zeros((8,8,7))
    x = 0
    y = 7 if isBlackTurn else 0

    yIncrement = -1 if isBlackTurn else 1

    for char in fen:
        if char.isnumeric():
            x += int(char)
        elif char == "/":
            x = 0
            y += yIncrement
        elif char == " ":
            break
        else:
            matrix[x,y,:] = pieceToVector(char, isBlackTurn)
            x += 1

    return matrix


def coordinateToVector(coordinate, isBlackTurn: bool = False):
    pieceMap = {
        "a": 0,
        "b": 1,
        "c": 2,
        "d": 3,
        "e": 4,
        "f": 5,
        "g": 6,
        "h": 7,
    }

    x = pieceMap[coordinate[0]]
    y = (int(coordinate[1]) - 1)

    if isBlackTurn:
        y = 7 - y

    vector = np.zeros(64)
    vector[(8*y)  +  x] = 1

    return vector

def uciToVector(uci, isBlackTurn: bool = False):
    fromVec = coordinateToVector(uci[:2], isBlackTurn)
    toVec = coordinateToVector(uci[2:], isBlackTurn)

    return np.kron(fromVec,toVec)
    

def getGameResult(headers):

    if headers["Termination"] == "Time forfeit":
        return None
    
    result = headers["Result"]

    if result == "1-0":
        return 1
    if result == "1/2-1/2":
        return 0.5
    if result == "0-1":
        return 0
    

def getAllPossibleMoves(board, isBlackTurn: bool = False):
    
    legalMoves = np.zeros(64*64)

    for move in board.legal_moves:
        legalMoves += uciToVector(move.uci(), isBlackTurn)

    return legalMoves




In [2]:
from chess import pgn
import pandas as pd
import random

# number of entries to take (this should be limited to what your computer's memory can handle)
gamesLimit = 1250000

#boards = []
#evaluations = []
#gameResults = []

turnOrientedBoards = []
moves = []


index = 0

# Need to flip vertical orientation of board to get more accurate moves, and also flip move vector
# that way directional pieces always move in the same direction
with open("lichess_db_standard_rated_2023-04.pgn","r") as chessData:

    game = pgn.read_game(chessData)

    while(game):
        if index > gamesLimit:
            break
        board = game.board()
        headers = game.headers
        #result = getGameResult(headers)

        if int(headers["BlackElo"]) < 2000 or int(headers["WhiteElo"]) < 2000:
            game = pgn.read_game(chessData)
            continue

        isBlackTurn = False
        turnNum = 0
        for move in game.mainline():

            # randomly don't take certain board states biasing later states 
            # because those have much more variation
            
            turnNum += 1
            if random.random() < (2/( turnNum+1 )) - 0.2:
                isBlackTurn = not isBlackTurn
                board.push(move.move)
                continue



            #boardMatrix = fenToMatrix(board.board_fen())

            turnBoard = fenToMatrix(board.board_fen(), isBlackTurn)
            uci = move.uci()
            moveVector = uciToVector(uci, isBlackTurn)

            #eval = move.eval()
            #if eval is None:
            #    break
            #else:
            #    eval = eval.white().score()

            isBlackTurn = not isBlackTurn

            #boards.append(boardMatrix)
            #evaluations.append(eval)
            #gameResults.append(result)
            turnOrientedBoards.append(turnBoard)
            moves.append(moveVector)
            index += 1

            board.push(move.move)

        game = pgn.read_game(chessData)
        


In [8]:
print(len(turnOrientedBoards))

2250050


In [3]:
#evaluationData = pd.DataFrame({"Board": boards, "Evaluation": evaluations, "Result": gameResults})
movingData = pd.DataFrame({"Board": turnOrientedBoards, "Move": moves})

In [12]:
evaluationData["Evaluation"] = evaluationData["Evaluation"].shift(1)
evaluationData["Result"] = evaluationData["Result"].shift(1)



                                               Board  Evaluation  Result
0  [[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0...         NaN     NaN
1  [[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0...        36.0     1.0
2  [[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0...        32.0     1.0
3  [[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0...         0.0     1.0
4  [[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0...        34.0     1.0


In [13]:
print(len(evaluationData))
evaluationData.dropna(inplace= True)
print(len(evaluationData))

1874105
1304021


In [4]:
#evaluationData.to_pickle("evaluation_data-2023-4.pkl")
movingData.to_pickle("moves_data-2023-4.pkl")

In [1]:
import pandas as pd
import numpy as np
movingData = pd.read_pickle("moves_data-2023-4.pkl")

boardData = np.array([board for board in movingData["Board"]])
moveData = np.array([move for move in movingData["Move"]])



In [5]:
boardData = np.array([board for board in movingData["Board"]])
moveData = np.array([move for move in movingData["Move"]])

In [6]:
import tensorflow as tf
from tensorflow import keras
from keras import layers


boardShape = (8,8,7)


movePredictorModel = keras.Sequential([
    layers.Input(boardShape),
    layers.Conv2D(56,3, activation="tanh"),
    layers.Conv2D(112,3, activation="tanh"),
    layers.Conv2D(224,3, activation="tanh"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(256,activation="tanh"),
    layers.Dense(256,activation="tanh"),
    layers.Dense(64*64, activation= "softmax")
])

print(movePredictorModel.summary())

movePredictorModel.compile(
    optimizer= "adam",
    loss= "categorical_crossentropy",
    metrics= ['accuracy']
)
#movePredictorModel.load_weights("movePredictor.h5")

2023-05-31 14:13:47.393300: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 6, 6, 56)          3584      
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 112)         56560     
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 2, 224)         226016    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 1, 224)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 224)               0         
                                                                 
 dense (Dense)               (None, 256)               57600     
                                                        

In [7]:
movePredictorModel.fit(
    x = boardData,
    y = moveData,

    epochs= 100,
    batch_size = 128,

    shuffle= True,
    validation_split= 0.2
)

movePredictorModel.save("movePredictor.h5")

Epoch 1/100
7813/7813 [==============================] - 320s 41ms/step - loss: 4.4276 - accuracy: 0.1353 - val_loss: 3.7977 - val_accuracy: 0.1729
Epoch 2/100
7813/7813 [==============================] - 302s 39ms/step - loss: 3.5600 - accuracy: 0.1922 - val_loss: 3.4795 - val_accuracy: 0.1999
Epoch 3/100
7813/7813 [==============================] - 302s 39ms/step - loss: 3.3296 - accuracy: 0.2127 - val_loss: 3.3488 - val_accuracy: 0.2096
Epoch 4/100
7813/7813 [==============================] - 302s 39ms/step - loss: 3.2039 - accuracy: 0.2248 - val_loss: 3.2800 - val_accuracy: 0.2165
Epoch 5/100
7813/7813 [==============================] - 304s 39ms/step - loss: 3.1249 - accuracy: 0.2332 - val_loss: 3.2450 - val_accuracy: 0.2214
Epoch 6/100
7813/7813 [==============================] - 308s 39ms/step - loss: 3.0655 - accuracy: 0.2398 - val_loss: 3.2288 - val_accuracy: 0.2231
Epoch 7/100
7813/7813 [==============================] - 308s 39ms/step - loss: 3.0234 - accuracy: 0.2446 - val_

KeyboardInterrupt: 

In [8]:
movePredictorModel.save("movePredictor.h5")

In [7]:


board = boardData[1]

print(board)
prediction = movePredictorModel.predict(np.asarray([board]))[0]

max = np.max(prediction)
print(np.max(prediction))

print(np.argmax(prediction))

prediction = (prediction > max/2)*prediction

np.set_printoptions(threshold=np.inf)

print(prediction)

[[[1. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [1. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 1. 0. 0. 0. 0. 0.]]

 [[0. 0. 1. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 1. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 1.]
  [0. 0. 0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]]

 [[0. 0. 0. 0. 1. 0. 1.]
  [0. 0. 0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.

In [1]:
import pandas as pd
import numpy as np
evaluationData = pd.read_pickle("evaluation_data-2023-4.pkl")

In [14]:
evaluationData["Result"] = evaluationData["Result"].shift(1)

In [15]:
print(len(evaluationData))

15558996


In [16]:
evaluationData.dropna(subset = ["Result"], inplace = True)

In [17]:
evaluationData.to_pickle("evaluation_data-2023-4.pkl")

In [15]:
boards = np.array([board for board in evaluationData["Board"]])
evaluations = np.array([eval for eval in evaluationData["Evaluation"]])


15265.0
-15265.0


In [32]:
import tensorflow as tf
from tensorflow import keras
from keras import layers


boardShape = (8,8,7)

boardEvaluationModel = keras.Sequential([
    layers.Input(boardShape),
    layers.Conv2D(28,3),
    layers.Conv2D(56,3),
    layers.Conv2D(112,3),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128),
    layers.Dense(64),
    layers.Dense(1)
])

print(boardEvaluationModel.summary())

boardEvaluationModel.compile(
    optimizer= "RMSprop",
    loss= "mean_squared_error",
    metrics= ['accuracy']
)

#boardEvaluationModel.load_weights("evaluator.h5")

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 6, 6, 28)          1792      
                                                                 
 conv2d_27 (Conv2D)          (None, 4, 4, 56)          14168     
                                                                 
 conv2d_28 (Conv2D)          (None, 2, 2, 112)         56560     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 1, 112)        0         
 2D)                                                             
                                                                 
 flatten_9 (Flatten)         (None, 112)               0         
                                                                 
 dense_31 (Dense)            (None, 128)               14464     
                                                      

In [35]:
boardEvaluationModel.fit(
    x = boards,
    y = evaluations,

    epochs= 10,
    batch_size = 128,

    shuffle= True,
    validation_split= 0.1
)

Epoch 1/10
9169/9169 [==============================] - 80s 9ms/step - loss: 149215.3594 - accuracy: 0.0294 - val_loss: 166069.5156 - val_accuracy: 0.0233
Epoch 2/10
9169/9169 [==============================] - 81s 9ms/step - loss: 147566.8125 - accuracy: 0.0297 - val_loss: 165794.5625 - val_accuracy: 0.0220
Epoch 3/10
9169/9169 [==============================] - 81s 9ms/step - loss: 146029.8281 - accuracy: 0.0296 - val_loss: 173772.6875 - val_accuracy: 0.0467
Epoch 4/10
 255/9169 [..............................] - ETA: 1:23 - loss: 137823.9531 - accuracy: 0.0301

KeyboardInterrupt: 

In [36]:
boardEvaluationModel.save("evaluator.h5")